## Transfer Learning using Pretrained Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

**For NPZ Dataset of product images**

In [ ]:
import numpy as np

# Load the .npz file
data = np.load(r'\mergedData.npz', allow_pickle= True)

# Check the keys in the .npz file to understand the structure
print("Keys in the .npz file:", data.keys())

# Assuming the keys are 'images' and 'labels'
Xdata = data['pixel_arrays']
ydata = data['entity_names']

# Display the shape of the data
print("X data shape:", Xdata.shape)
print("y data shape:", ydata.shape)

# Optionally, split the dataset into training and testing sets manually
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xdata, ydata, test_size=0.2, random_state=42)

print("Training set shapes:", X_train.shape, y_train.shape)
print("Test set shapes:", X_test.shape, y_test.shape)


Keys in the .npz file: KeysView(NpzFile 'D:\\minorImage\\dataNPZ\\mergedData.npz' with keys: pixel_arrays, entity_names)
X data shape: (4500, 512, 512)
y data shape: (4500, 7)
Training set shapes: (3600, 512, 512) (3600, 7)
Test set shapes: (900, 512, 512) (900, 7)


In [4]:
X_train[0].shape

(512, 512)

In [ ]:
print("Train set shapes:", X_train.shape, y_train.shape)

Test set shapes: (900, 512, 512) (900, 7)


In [ ]:
print("Test set shapes:", X_test.shape, y_test.shape)

In [7]:
y_test[0]

array([1., 0., 0., 0., 0., 0., 0.])

**For Adding the third dimension (color channel)**

In [ ]:
# change the type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# convert data to 3 channels for the pretrained model

X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

In [ ]:
print("Train set shapes:", X_train.shape, y_train.shape)

((3600, 512, 512, 3), (900, 512, 512, 3))

In [11]:
X_train[0].shape

(512, 512, 3)

In [ ]:
# data augmentation with generator (creates different forms of images to increase data size)
train_generator = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 40, # rotates the image by 40 degree
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest' #to fill in missing pixel
)

val_generator = ImageDataGenerator(rescale = 1./255) #for greyscale (no augmentation for validation data)

train_iterator = train_generator.flow(X_train, y_train, batch_size=4, shuffle=True)

val_iterator = val_generator.flow(X_test, y_test, batch_size=4, shuffle=False)

**Model Creation**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
# add the pretrained model
model.add(EfficientNetB0(input_shape=(512, 512, 3), include_top=False, pooling='avg', weights='imagenet'))
# include_top=False removes the prediction layert, otherwise it will use the set 1000 classes

# add fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5)) 
model.add(Dense(7, activation='softmax'))

# set ResNet layers to not be trainable
model.layers[0].trainable = False # so the first layer (ResNet) will not be trained

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 1280)           │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,709,034 (17.96 MB)

 Trainable params: 659,463 (2.52 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
#train the model

model.fit(train_iterator, epochs=10, validation_data = val_iterator)

Epoch 1/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 498s 544ms/step - accuracy: 0.6961 - loss: 1.1798 - val_accuracy: 0.7078 - val_loss: 1.0793
Epoch 2/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 476s 527ms/step - accuracy: 0.6935 - loss: 1.1456 - val_accuracy: 0.7078 - val_loss: 1.0911
Epoch 3/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 477s 529ms/step - accuracy: 0.6970 - loss: 1.1412 - val_accuracy: 0.7078 - val_loss: 1.0756
Epoch 4/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 476s 528ms/step - accuracy: 0.6938 - loss: 1.1349 - val_accuracy: 0.7078 - val_loss: 1.0727
Epoch 5/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 475s 527ms/step - accuracy: 0.6850 - loss: 1.1635 - val_accuracy: 0.7078 - val_loss: 1.0860
Epoch 6/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 475s 527ms/step - accuracy: 0.6955 - loss: 1.1320 - val_accuracy: 0.7078 - val_loss: 1.0776
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 476s 528ms/step - accuracy: 0.6988 - loss: 1.1024 - val_accuracy: 0.7078 - val_loss: 1.0912
Epoch 8/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 476s 528ms/step - accuracy: 0.6894 -

In [ ]:
import numpy as np

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(val_iterator)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 77s 342ms/step - accuracy: 0.7068 - loss: 1.0696
Test Loss: 1.0718724727630615
Test Accuracy: 0.7077777981758118


In [ ]:
# Save the model in the native Keras format
model.save(r'\website\savedModel\model22f.keras')
